# Scraping des offres sur le site `marchespublics.sn`

Dans ce notebook, nous allons utiliser Selenium pour scraper les offres publiées sur le site [marchespublics.sn](http://www.marchespublics.sn/index.php?option=com_loffres&Itemid=104).

In [1]:
try:
    get_ipython().run_line_magic('pip', 'install selenium')
    get_ipython().run_line_magic('pip', 'install webdriver-manager')
    get_ipython().run_line_magic('pip', 'install pandas')
except NameError:
    pass

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime
import json

In [3]:
# Configurer Selenium avec ChromeDriver
options = Options()
options.add_argument('--headless')  # Exécuter en mode sans tête (sans interface graphique)
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

In [4]:
# Configuration du driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [5]:
# URL des offres "Prestation de services"
url = "http://www.marchespublics.sn/index.php?option=com_loffres&task=view&idcat=002&Itemid=104&gestion=2025&statut=1"
driver.get(url)


In [6]:
# definir l'année courante
current_year = datetime.now().year
# Trouver le tableau contenant les offres
rows = driver.find_elements(By.CSS_SELECTOR, "table.content_table tbody tr")[1:]  # pour sauter l'entête


In [7]:
data = []

for row in rows[1:]:
    try:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) < 6:
            continue

        # ✅ Tenter de récupérer le lien AVANT de continuer
        try:
            detail_td = cols[5]
            div = detail_td.find_element(By.TAG_NAME, "div")
            a_tags = div.find_elements(By.TAG_NAME, "a")
            if not a_tags:
                continue  # Pas de lien => ce n'est pas une ligne d'offre
            detail_link = a_tags[0].get_attribute("href")
        except:
            continue  # Évite les erreurs sur les lignes non valides

        # ✅ Extraire les autres infos de la ligne
        ref = cols[0].text.strip() or "NA"
        objet = cols[1].text.strip() or "NA"
        autorite = cols[2].text.strip() or "NA"
        publie_le = cols[3].text.strip() or "NA"
        date_limite = cols[4].text.strip() or "NA"

        # ✅ Ouvrir le lien dans un nouvel onglet
        driver.execute_script("window.open(arguments[0]);", detail_link)
        driver.switch_to.window(driver.window_handles[1])

        try:
            table_elem = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table.no-style"))
            )
            td_elem = table_elem.find_element(By.TAG_NAME, "td")
            detail_text = td_elem.text.strip() or "NA"
        except Exception as e:
            print(f"⚠️ Détail non trouvé pour {ref} : {e}")
            detail_text = "NA"

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        data.append([ref, objet, autorite, publie_le, date_limite, detail_text])

    except Exception as e:
        print(f"❌ Erreur sur l’offre {ref if 'ref' in locals() else 'inconnue'} : {e}")
        continue

driver.quit()


In [8]:
print("Nombre d'offres récupérées :", len(data))

Nombre d'offres récupérées : 6


In [9]:
# le dataframe
df = pd.DataFrame(data, columns=["Référence", "Objet", "Autorité", "Publié le", "Date limite", "Détail"])
df

,Référence,Objet,Autorité,Publié le,Date limite,Détail
0,F_FADCL_006,Acquisition Catalogues et imprimés publicitair...,Fonds d'Appui au Développement du Contenu loca...,12/05/2025,23/05/2025,NA
1,S_DST_007,Location de véhicules de pompage,Ville de Pikine,03/05/2025,19/05/2025,Avis de la demande de renseignement et de prix...
2,S_FADCL_005,hébergements des plateformes mines et hydrocar...,Fonds d'Appui au Développement du Contenu loca...,09/05/2025,22/05/2025,NA
3,S_DCH_125,sélection d'un prestataire pour l'organisation...,Port Autonome de Dakar (PAD),18/04/2025,21/05/2025,PORT AUTONOME DE DAKAR\n==================\nAv...
4,S_CPM_018,Entretein et réparation de véhicules 2 lots,NA,18/04/2025,19/05/2025,Avis d'Appel d'offres (AA0)\nMinistère de l'En...
5,S_CQHSE_110,Sélection d’un prestataire pour l’accompagneme...,Port Autonome de Dakar (PAD),14/05/2025,14/05/2025,NA


In [10]:
offres_liste = df.to_dict(orient='records')
offres_liste

[{'Référence': 'F_FADCL_006',
  'Objet': "Acquisition Catalogues et imprimés publicitaires (travaux d'imprimerie)",
  'Autorité': "Fonds d'Appui au Développement du Contenu local (FADCL)",
  'Publié le': '12/05/2025',
  'Date limite': '23/05/2025',
  'Détail': 'NA'},
 {'Référence': 'S_DST_007',
  'Objet': 'Location de véhicules de pompage',
  'Autorité': 'Ville de Pikine',
  'Publié le': '03/05/2025',
  'Date limite': '19/05/2025',
  'Détail': 'Avis de la demande de renseignement et de prix à compétition ouverte (ADRPCO)\nVille de Pikine\nAvis de la DRPCO n° S_DST_007/2025\nCet Avis d\'appel public à concurrence fait suite à l\'Avis Général de Passation des Marchés paru dans dans le journal "SUD QUOTIDIEN" n°9464 du jeudi 12 décembre 2024.\nLa Ville de Pikine a obtenu dans le cadre de son budget 2025 des fonds et a l\'intention d\'utiliser une partie de ces fonds pour effectuer des paiements au titre du Marché n°S-DST-007/2025 relatif à la location de véhicules de pompage.\nLa Ville de

In [16]:
offres_liste[0]

{'Référence': 'F_FADCL_006',
 'Objet': "Acquisition Catalogues et imprimés publicitaires (travaux d'imprimerie)",
 'Autorité': "Fonds d'Appui au Développement du Contenu local (FADCL)",
 'Publié le': '12/05/2025',
 'Date limite': '23/05/2025',
 'Détail': 'NA'}

In [18]:
# Convertir le dictionnaire en format texte
offres_texte = json.dumps(offres_liste, indent=4, ensure_ascii=False)
print(offres_texte)

[
    {
        "Référence": "F_FADCL_006",
        "Objet": "Acquisition Catalogues et imprimés publicitaires (travaux d'imprimerie)",
        "Autorité": "Fonds d'Appui au Développement du Contenu local (FADCL)",
        "Publié le": "12/05/2025",
        "Date limite": "23/05/2025",
        "Détail": "NA"
    },
    {
        "Référence": "S_DST_007",
        "Objet": "Location de véhicules de pompage",
        "Autorité": "Ville de Pikine",
        "Publié le": "03/05/2025",
        "Date limite": "19/05/2025",
        "Détail": "Avis de la demande de renseignement et de prix à compétition ouverte (ADRPCO)\nVille de Pikine\nAvis de la DRPCO n° S_DST_007/2025\nCet Avis d'appel public à concurrence fait suite à l'Avis Général de Passation des Marchés paru dans dans le journal \"SUD QUOTIDIEN\" n°9464 du jeudi 12 décembre 2024.\nLa Ville de Pikine a obtenu dans le cadre de son budget 2025 des fonds et a l'intention d'utiliser une partie de ces fonds pour effectuer des paiements au tit